<a href="https://colab.research.google.com/github/Asare-Obed/WorkOrder-Automation/blob/main/work_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALLING AND IMPORTING MODULES**

In [ ]:
#!pip install docxtpl

In [ ]:
import pandas as pd
import os
from docxtpl import DocxTemplate
from datetime import datetime
from collections import defaultdict
from pprint import pprint

# **USER CONFIGURATION AND DATA PATHS**

In [ ]:
WORK_PLAN_FILE = "/content/drive/MyDrive/Work Order_May22-June19_2025.xlsx"
LOOKUP_FILE = "/content/drive/MyDrive/KoboTestTable for WO Automation.xlsx"
TEMPLATE_MAP = {
    "Planting": "/content/drive/MyDrive/Planting-WorkOrderTemplate.docx",
    "Pitting": "/content/drive/MyDrive/Establishment_Holing-WorkOrderTemplate.docx",
    "Establishment Clearing": "/content/drive/MyDrive/Establishment_Clearing-WorkOrderTemplate.docx"}
OUTPUT_FOLDER = "/content/drive/MyDrive/rbgh_automations/generated_workorder"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
df = pd.read_excel(WORK_PLAN_FILE)

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print(df.columns.tolist())

['s/n', 'week_number', 'date', 'section', 'operation', 'work_location', 'assigned_team', 'work_quantity_(ha)', 'forester_name', 'team_leader', 'work_order_id']


In [ ]:
df = df[df['work_order_id'].notna()]

In [ ]:
df['work_order_id'] = df['work_order_id'].astype(str).str.strip().str.lower()
df['work_order_id'] = df['work_order_id'].str.replace("wo-", "wo_")  # Normalize to match lookup

#**LOAD KOBO LOOKUP TABLE**

**If the data has already been cleaned manually or it's a single sheet in the spreadsheet then this code block will be used**



```
lookup_df = pd.read_excel(LOOKUP_FILE)
lookup_df.columns = lookup_df.columns.str.strip().str.lower().str.replace(" ", "_")
print(lookup_df.columns.tolist())
lookup_dict = (
    lookup_df
    .groupby("work_order_id")
    .apply(lambda g: g.to_dict(orient="records"))
    .to_dict()
)
```



These Kobo exports typically contain:
* A main sheet with form submissions (each row = one record)
* One or more sub-sheets for repeat groups (like worker details, plots, etc.)

So if your kobo data has a similar structure then this should clean and restructure the tables in a single table.

In this code the lookup file contains the kobo data file filled by Silviculture for their daily input per worker per activity.

In [ ]:
# Load all sheets
kobo_file = pd.read_excel(LOOKUP_FILE, sheet_name=None)

# Extract main sheet and repeat group
main_df = list(kobo_file.values())[0]
repeat_df = list(kobo_file.values())[1]  # change index if needed

# Standardize column names
main_df.columns = main_df.columns.str.strip().str.lower().str.replace(" ", "_")
repeat_df.columns = repeat_df.columns.str.strip().str.lower().str.replace(" ", "_")
print("Main_df_columns=",main_df.columns.tolist())
print("Repeat_df_columns=",repeat_df.columns.tolist())

Main_df_columns= ['qc_attempt', 'date_of_qc', 'work_order_number', 'work_order_id', 'start_date_of_activity', 'end_date_of_activity', 'technical_officer', 'qc_officer', 'forest_reserve', 'compartment_id', 'sub-compartment', 'plot_number', 'gps_point', '_gps_point_latitude', '_gps_point_longitude', '_gps_point_altitude', '_gps_point_precision', 'land_cover', 'photo_north', 'photo_north_url', 'photo_east', 'photo_east_url', 'photo_south', 'photo_south_url', 'photo_west', 'photo_west_url', '_id', '_uuid', '_submission_time', '_validation_status', '_notes', '_status', '_submitted_by', '__version__', '_tags', '_index']
Repeat_df_columns= ['worker_name', 'activity_id', 'activity', 'work_quantity_ha', 'work_quantity_m', 'slashing/bush_clearance_height_of_weeds_</=_15cm', 'slashing/bush_clearing_completeness', 'all_weeds_treated', 'marking_and_pitting_-_correct_spacing', 'number_of_pits', 'number_of_pits_with_unfirmed_soil_tilth', 'number_of_pits_with_niche_depth_and_width_>/=_30cm', 'all_larg

In [ ]:
# Merge based on _index from main table and parent_index from repeat table
lookup_df = repeat_df.merge(
    main_df,
    left_on='_parent_index',
    right_on='_index',
    how='left'
)

In [ ]:
lookup_df['work_order_id'] = lookup_df['work_order_id'].astype(str).str.strip().str.lower()
lookup_df['work_order_id'] = lookup_df['work_order_id'].str.replace("wo-", "wo_")  # Normalize to match work plan

Run this portion to check the merged sheets.

In [ ]:
output_path = os.path.join(OUTPUT_FOLDER, "merged_lookup_export2.xlsx")
lookup_df.to_excel(output_path, index=False)
print(f"✅ Exported to {output_path}")

✅ Exported to /content/drive/MyDrive/rbgh_automations/generated_workorder/merged_lookup_export2.xlsx


In [ ]:
# Now build lookup_dict grouped by wo_id
lookup_dict = (
    lookup_df
    .dropna(subset=['work_order_id'])  # ensure WO ID is available
    .groupby("work_order_id")
    .apply(lambda g: g.to_dict(orient="records"))
    .to_dict()
)

/tmp/ipython-input-1966069555.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.to_dict(orient="records"))


In [ ]:
print("WO IDs found in work plan:", df['work_order_id'].unique())
print("WO IDs found in lookup dict:", list(lookup_dict.keys())[:5], "...")

WO IDs found in work plan: ['wo_4' 'wo_5' 'wo_6' 'wo_7' 'wo_8' 'wo_9' 'wo_10' 'wo_11' 'wo_12' 'wo_3'
 'wo_13' 'wo_14' 'wo_15']
WO IDs found in lookup dict: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5'] ...




```
for wo_id, entries in list(lookup_dict.items())[:3]:  # show first 3 WO IDs
    print(f"\nWork Order ID: {wo_id}")
    for entry in entries:
        pprint(entry)
```



GROUP BY WORK ORDER ID

In [ ]:
for wo_id, group in df.groupby("work_order_id"):
    row = group.iloc[0]
    activity = row['operation'].strip()

    # Pick correct template
    matched_template = None
    for key in TEMPLATE_MAP:
        if key.lower() in activity.lower():
            matched_template = TEMPLATE_MAP[key]
            break

    if not matched_template or not os.path.exists(matched_template):
        print(f"⚠️ No template found for activity: {activity}, WO ID: {wo_id}")
        continue

    print(f"🔎 Attempting lookup with key: {wo_id}")
    print(f"🔎 Available keys: {list(lookup_dict.keys())[:5]}...")  # only show first few

    workers_info = lookup_dict.get(wo_id, [])
    print(f"🧩 Matching WO ID: {wo_id} → Found {len(workers_info)} workers")

    if not workers_info:
        print(f"  ⚠️ Skipping {wo_id} (no worker data)")
        continue

    for person in workers_info:
        doc = DocxTemplate(matched_template)

        issue_date = row['date']
        due_date = group['date'].max()

        target_areas = []
        for _, r in group.iterrows():
            plot = r['work_location']
            target_areas.append({
                "reserve": "Chai River",
                "compartment": plot[:2],
                "sub_compartment": plot,
                "plot_no": plot,
                "area_ha": r['work_quantity_(ha)']
            })

        context = {
            "worker": person.get("worker", "Unnamed"),
            "activity": activity,
            "work_order_no": wo_id.upper().replace("_", "-"),
            "date_issued": issue_date.strftime("%d/%m/%Y"),
            "due_date": due_date.strftime("%d/%m/%Y"),
            "technical_officer": row.get("forester_name", "John Kodua"),
            "target_areas": target_areas,
            "activity_id": person.get("activity_id", "UNK"),
            "mandays_ha": person.get("mandays_per_ha", "1.0"),
            "rate": person.get("rate", "0.51"),
            "total_amount": person.get("amount", "Auto"),
            "qc_result": person.get("qc_result", "Pass"),
            "total_seedlings": person.get("seedlings", "1945"),
        }

        doc.render(context)

        # Format filename: WOID_Date_WorkerName.docx
        formatted_date = due_date.strftime("%Y-%m-%d")
        worker_name = person.get("worker", "Unnamed").lower().replace(" ", "_")
        filename = f"{wo_id.upper()}_{worker_name}_{formatted_date}.docx"
        output_docx = os.path.join(OUTPUT_FOLDER, filename)
        doc.save(output_docx)

        # Optional: Convert to PDF if needed
        #try:
            #import shutil
            #if shutil.which("libreoffice"):
                #import subprocess
                #subprocess.run([
                    #"libreoffice", "--headless", "--convert-to", "pdf", output_path, "--outdir", OUTPUT_FOLDER
                #])
            #else:
                #print(f"⚠️ LibreOffice not found. Skipping PDF export for {filename}")
        #except Exception as e:
            #print(f"❌ PDF export error for {filename}: {e}")

print("✅ Work orders generated.")


🔎 Attempting lookup with key: wo_10
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_10 → Found 12 workers
🔎 Attempting lookup with key: wo_11
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_11 → Found 7 workers
🔎 Attempting lookup with key: wo_12
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_12 → Found 0 workers
  ⚠️ Skipping wo_12 (no worker data)
🔎 Attempting lookup with key: wo_13
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_13 → Found 0 workers
  ⚠️ Skipping wo_13 (no worker data)
🔎 Attempting lookup with key: wo_14
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_14 → Found 11 workers
🔎 Attempting lookup with key: wo_15
🔎 Available keys: ['wo_10', 'wo_11', 'wo_14', 'wo_3', 'wo_5']...
🧩 Matching WO ID: wo_15 → Found 0 workers
  ⚠️ Skipping wo_15 (no worker data)
🔎 Attempting lookup with key: